In [1]:
import pandas as pd
import numpy as np
import nltk
import nltk.sentiment.util
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import string
import math

In [2]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [56]:
df = pd.read_csv('../../amazon_reviews_us_Grocery_v1_00_sentiment.csv').sample(100000)

df = df.drop(["marketplace", "customer_id", "product_parent", "product_category", "review_date"], axis=1)

df = df[~df['review_body'].isnull()]

df['review_body'] = df['review_headline'] + '. ' + df['review_body']

df["health_hazard"] = np.nan

df = df.set_index("review_id")

In [57]:
df

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard
review_id,,,,,,,,,,,,
RLSO7CW3S1GOT,B004VD6S8K,Barr's Irn-Bru,5,0,0,N,Y,In Bru,In Bru. My hubby loved the Iron Bru he said th...,0.300000,0.575000,NaN
RYRKM8U8HPUOK,B00C75UT0C,"Soul Sprout, by Two Moms Sprouted Grain Free C...",5,1,1,N,Y,Great by itself,Great by itself. This is the most unique grano...,0.556250,0.641667,NaN
R1R7UU0N0KGXX3,B00FPLGA6C,"Chef Boyardee Pepperoni Pizza Maker Kit, 31.85...",5,0,0,N,Y,Five Stars,Five Stars. Lots of fun!,0.375000,0.200000,NaN
RITECYC69L97R,B0004N0T86,McCormick Perfect Pinch Lemon & Pepper Seasoni...,5,1,1,N,Y,Lemon....yummy!,Lemon....yummy!. This lemon pepper has a reall...,0.500000,0.400000,NaN
R2DOF1LVJGS18M,B00M0NEK3S,Starbucks Variety Coffee K-Cup Featuring 3 Dar...,5,0,0,N,Y,Delicious!,Delicious!. Nice variety of dark roasts...deli...,0.103125,0.350000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
RF9EF902NTRZR,B007K6SSTC,"Grove Square Cappuccino, Single Serve Cup for ...",4,0,0,N,Y,it is ok,"it is ok. it tastes ok, but sometimes if you m...",0.183333,0.316667,NaN
RI76ISLK745TE,B00C3M61AQ,"Nestle Crunch Girl Scout Candy Bars, Thin Mint...",5,5,5,N,N,A great minty treat,A great minty treat. While these are made by N...,0.210000,0.285000,NaN
R7ABK0IR8WQ05,B001BMDF1M,"Health Valley Health Valley Organic Soup, Vege...",5,0,0,N,Y,no-salt healthy valley vegetable soup,no-salt healthy valley vegetable soup. This is...,0.450000,0.283333,NaN


In [58]:
neg_reviews = df.query("star_rating < 4", engine="python")

In [59]:
stop_words = stopwords.words('english')
stop_words.extend(['br'])
translator=str.maketrans('','',string.punctuation)

#neg_reviews["tokened_review"] = np.nan
all_tokens = []
#neg_reviews.astype({"tokened_review": 'object'}).dtypes
for index, row in neg_reviews.iterrows():
    text = row["review_body"].lower()
    text = text.translate(translator)
    text = word_tokenize(text)
    new_text = []
    for token in text:
        token = stemmer.stem(token)
        if token not in stop_words:
            new_text.append(token)
    all_tokens.append(new_text)
neg_reviews["tokened_review"] = all_tokens

<ipython-input-59-eba5c76ea0a6>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["tokened_review"] = all_tokens


In [60]:
neg_reviews

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
RT2Y92DQLOI7V,B000LKU4XM,Road's End Organics Gluten Free Mac & Chreese ...,1,0,1,N,N,So bad!,So bad!. I was shocked at how bad this product...,-0.117284,0.297840,NaN,"[bad, wa, shock, bad, thi, product, look, tast..."
R2KM5IIC4MULJB,B00C0WHDME,"Popcorn Indiana Movie Theater Popcorn, 5.5 Ounce",1,0,3,N,N,The Worst,The Worst. While I admire the company itself a...,-0.433333,0.565278,NaN,"[worst, admir, compani, ingredi, pride, selv, ..."
R3NNXIDO22ST3X,B00HKDKUO6,"Louisiana Purchase Red Beans and Rice Cup, 2 O...",1,0,1,N,Y,This is not good stuff. I could get some red b...,This is not good stuff. I could get some red b...,0.008333,0.550000,NaN,"[thi, good, stuff, could, get, red, bean, thi,..."
R3SPI91Y0N24MC,B004EKHMXU,Healthworks Cacao Nibs Parent,2,0,0,N,Y,Poor taste!,Poor taste!. I've eaten many raw cacao nibs be...,0.200427,0.365812,NaN,"[poor, tast, ive, eaten, mani, raw, cacao, nib..."
R25DJYDTCXQZ25,B0055Z8ROQ,The Sprout House Certified Organic Non-gmo Spr...,3,2,2,N,N,Disappointed,Disappointed. Only half of the seeds actually ...,-0.174583,0.463333,NaN,"[disappoint, onli, half, seed, actual, sprout,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R33FB4DK9QBJ4W,B004MAV8M8,Sierra Tea Organic Supermicro Grounded Matcha ...,2,0,0,N,Y,not good flavor,not good flavor. bitter. just not a very enjoy...,-0.089231,0.246154,NaN,"[good, flavor, bitter, veri, enjoy, matcha, un..."
R3CUH1YEGV4MPQ,B004V6A9SC,"Newman's Own Con Queso Salsa, 16-Ounce (Pack o...",2,40,55,N,N,So I did find good use for it after all,So I did find good use for it after all. I bou...,0.152000,0.388000,NaN,"[find, good, use, bought, thi, catfish, bait, ..."
R23FZP3HUPJQTT,B00KOPGQRW,Tabasco Brand Buffalo Hot Sauce - by McIlhenny...,1,0,0,N,Y,One Star,One Star. This stuff is terrible!,-1.000000,1.000000,NaN,"[one, star, thi, stuff, terribl]"


In [61]:
hazardous_words=["rotten", "mouldy", "moldy", "mold", "mould", "sick", "dangerous", "diarrhea", "poisoning", "stale", "contaminate",'disease', 'disgusting', 'hospital', 'poisoning', 'hazardous', 'ill', 'mold','moldy', 'mildew', 'dangerous' 'rancid', 'smelly', 'stale', 'unhealthy', 'sick','expired','expiration']
for word in hazardous_words:
    neg_reviews.loc[[stemmer.stem(word) in tokened_review for tokened_review in neg_reviews["tokened_review"]], "health_hazard"] = 1
    

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [62]:
num_labeled = len(neg_reviews.query("health_hazard == 1", engine="python"))
num_labeled

1989

In [63]:
non_hazardous = neg_reviews.query("health_hazard != 1", engine="python").sample(3*num_labeled)
for i in list(non_hazardous.index):
    neg_reviews.loc[i, "health_hazard"] = 0

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [64]:
labeled = neg_reviews.query("health_hazard == 0 or health_hazard == 1", engine="python")
reviews = labeled.loc[:, 'tokened_review'].values
y = labeled.loc[:, 'health_hazard'].values

In [65]:
pprint.pprint(reviews)

array([list(['bad', 'wa', 'shock', 'bad', 'thi', 'product', 'look', 'tast', 'noodl', 'never', 'got', 'soft', 'cook', '15', 'minut', 'suggest', '610', 'think', 'hard', 'tast', 'stale', 'far', 'befor', 'expir', 'date', 'sauc', 'oh', 'sauc', 'wa', 'grey', 'gritti', 'tast', 'like', 'rancid', 'chicken', 'broth', 'kid', 'took', 'one', 'bite', 'refus', 'eat', 'anoth', 'stomach', 'kid', 'serious', 'wa', 'gross']),
       list(['poor', 'tast', 'ive', 'eaten', 'mani', 'raw', 'cacao', 'nib', 'befor', 'par', 'strong', 'moldi', 'tast', 'mani', 'nib', 'mayb', 'wa', 'fluke']),
       list(['disappoint', 'onli', 'half', 'seed', 'actual', 'sprout', 'vital', 'wa', 'good', 'growth', 'wa', 'veri', 'slow', 'think', 'seed', 'sit', 'shelf', 'long']),
       ...,
       list(['mostli', 'broken', 'piec', 'want', 'fool', 'think', 'wa', 'eat', 'pretzel', 'stick', 'wa', 'imposs', 'bag', 'broken', 'piec']),
       list(['buy', 'horribl', 'horribl', 'bad', 'got', 'didnt', 'mark', 'perish', 'packag', 'wa', 'instruct

In [66]:
for i in range(len(reviews)):
    reviews[i] = ' '.join(reviews[i])

In [67]:
reviews

array(['bad wa shock bad thi product look tast noodl never got soft cook 15 minut suggest 610 think hard tast stale far befor expir date sauc oh sauc wa grey gritti tast like rancid chicken broth kid took one bite refus eat anoth stomach kid serious wa gross',
       'poor tast ive eaten mani raw cacao nib befor par strong moldi tast mani nib mayb wa fluke',
       'disappoint onli half seed actual sprout vital wa good growth wa veri slow think seed sit shelf long',
       ...,
       'mostli broken piec want fool think wa eat pretzel stick wa imposs bag broken piec',
       'buy horribl horribl bad got didnt mark perish packag wa instruct leav outsid noon answer',
       'ok broth wont find ani chunk beef thi good broth thi microwav contain'],
      dtype=object)

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
matrix = TfidfVectorizer(max_features=200)
X = matrix.fit_transform(reviews).toarray()
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.42352139, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [69]:
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [70]:
from sklearn.naive_bayes import GaussianNB
classifier_g = GaussianNB()

from sklearn.linear_model import LogisticRegression

classifier_l = LogisticRegression()

classifier_g.fit(X_train, y_train)
classifier_l.fit(X_train, y_train)

# Predict Class
y_pred_g = classifier_g.predict(X_test)
y_pred_l = classifier_l.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy_g = accuracy_score(y_test, y_pred_g)
accuracy_l = accuracy_score(y_test, y_pred_l)
print(accuracy_g)
print(accuracy_l)

0.9411764705882353
0.9316239316239316


In [48]:
trial = neg_reviews.sample(100)

In [49]:
matrix.get_feature_names()

['actual',
 'ad',
 'add',
 'almost',
 'also',
 'amazon',
 'ani',
 'anoth',
 'anyth',
 'arriv',
 'aw',
 'away',
 'back',
 'bad',
 'bag',
 'bar',
 'bean',
 'becaus',
 'befor',
 'best',
 'better',
 'bit',
 'bitter',
 'bottl',
 'bought',
 'box',
 'brand',
 'buy',
 'came',
 'candi',
 'cant',
 'chip',
 'chocol',
 'coffe',
 'come',
 'compani',
 'contain',
 'cooki',
 'could',
 'cup',
 'date',
 'day',
 'didnt',
 'differ',
 'disappoint',
 'disgust',
 'doe',
 'doesnt',
 'dont',
 'dri',
 'drink',
 'eat',
 'enjoy',
 'enough',
 'even',
 'ever',
 'everi',
 'expect',
 'expens',
 'expir',
 'feel',
 'find',
 'fine',
 'first',
 'flavor',
 'food',
 'found',
 'free',
 'fresh',
 'fruit',
 'get',
 'gift',
 'give',
 'go',
 'good',
 'got',
 'great',
 'green',
 'ha',
 'half',
 'hard',
 'high',
 'hope',
 'horribl',
 'hot',
 'howev',
 'id',
 'ill',
 'im',
 'ingredi',
 'item',
 'ive',
 'kind',
 'know',
 'label',
 'last',
 'less',
 'like',
 'littl',
 'local',
 'look',
 'lot',
 'love',
 'made',
 'make',
 'mani',
 'm

In [50]:
trial = neg_reviews
trial_x = trial.loc[:,"tokened_review"].values
for i in range(len(trial_x)):
    trial_x[i] = ' '.join(trial_x[i])
trial_X = matrix.fit_transform(trial_x).toarray()
trial_y = classifier.predict(trial_X)
trial_y

NameError: name 'classifier' is not defined

In [ ]:
neg_reviews["health_hazard"] = trial_y

In [ ]:
neg_reviews

In [ ]:
dangerous = neg_reviews.query("health_hazard == 1", engine="python")
dangerous

In [ ]:
#LDA
data_filtered = dangerous['review_body'].values
pprint.pprint(data_filtered)

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = sentence.translate(translator)
        yield(nltk.word_tokenize(str(sentence)))

data_words = list(sent_to_words(data_filtered))
clean_data = []
for i in range(len(data_words)):
    clean_review = []
    for j in range(len(data_words[i])):
        if data_words[i][j] not in stop_words:
            clean_review.append(data_words[i][j])
    clean_data.append(clean_review)

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(clean_data)
# Create Corpus
texts = clean_data
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
pprint.pprint(corpus[:5])

In [ ]:
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       minimum_probability=0.1,
                                       workers=3,
                                       passes=2)
# Print the Keyword in the 10 topics
pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
#from itertools import chain
#lda_corpus = lda_model[corpus]

# Find the threshold, let's set the threshold to be 1/#clusters,
# To prove that the threshold is sane, we average the sum of all probabilities:
#scores = list(chain(*[[score for topic_id,score in topic] for topic in [doc for doc in lda_corpus]]))
#threshold = sum(scores)/len(scores)


In [ ]:
import pickle 
import pyLDAvis
import pyLDAvis.gensim_models
data_vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

pyLDAvis.display(data_vis)